<a href="https://colab.research.google.com/github/riverdogcabin/PSDS4900/blob/main/wunderground_weekly_pull.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import json
from pprint import pprint
import pandas as pd
from datetime import datetime
from time import sleep

In [ ]:
# Set up API call URL
# weatherreadings = {}
url = 'https://api.weather.com/v2/pws/observations/hourly/7day?stationId={stationid}&format=json&units={units}&apiKey={apikey}&numericPrecision={numeric_precision}'
# url = "https://api.weather.com/v2/pws/observations/current?stationId=REPLACESTATIONID&format=json&units=REPLACEUNITS&numericPrecision=decimal&apiKey=REPLACEAPI"


In [ ]:
# Get Config Data from config.json
with open('/content/drive/MyDrive/Capstone/PSDS4900/config.json') as configuration:
  config = json.load(configuration).get('WU') #config for WeatherUnderground
  apikey = config["apikey"]
  my_stationid = config["stationid"]
  units = config["units"]
  numeric_precision = config["numeric_precision"]

In [ ]:
url = url.format(stationid = '{stationid}',units=units,apikey=apikey,numeric_precision=numeric_precision)
# print(url)
station_file = '/content/drive/MyDrive/Capstone/PSDS4900/wu_stations.txt'
drop_columns = ['lat','lon','units','tz','obsTimeUtc','obsTimeLocal']

In [ ]:
# JSON comes back with measurements nested inside 'metrics', need to flatten
def flatten_wu_result(wu_result,units='metric'):
  try:
    observation = wu_result.pop(units)
    wu_result.update(observation)
    wu_result.update({'units':units})
  except:
    return wu_result

In [ ]:
# function to get the JSON data from Weather Underground
def get_wu_data(stationid,url):
  url = url.format(stationid=stationid)
  print("Retrieving data for " + stationid,end=': ')
  response = requests.get(url) # need to add error handling for invalid data

  if response.status_code == 200: # If the response code is 200 (sucessful)
      print("Success")
      weatherdata = response.json()
      # pprint(weatherdata)
      for observation in weatherdata.get('observations'):
        flatten_wu_result(observation)
      return weatherdata.get('observations')
  elif response.status_code == 204: # 204 returns a blank response so inform the user
      print("API Error 204: No Data Found for specific query.")
      exit()

  elif response.status_code >200 & response.status_code <1000: # Handles all other error code numbers
      result = response.json()
      try:
          print("API Error " + str(response.status_code) + ": " + result["errors"][0]["error"]["message"])
      except: # sometimesthe error messae is in a different nest
          print("API Error " + str(response.status_code) + ": " + result["errors"][0]["message"])
      exit()

  else: # Catch any other errors
      print("Unspecified error")
      print(response.text)

In [ ]:
def get_stations(stationfile):
  station_list = []
  with open(stationfile) as infile:
    for l in infile.readlines():
      if(l.find('#') == -1):
        station_list.append(l.strip())
  return station_list

In [ ]:
station_list = get_stations(station_file)
# print(station_list)
# len(weatherdata.get('observations'))

In [ ]:
observations_df = pd.DataFrame()
for station in station_list:
  observations_df = observations_df.append(pd.DataFrame(get_wu_data(station,url)))
  sleep(1)

Retrieving data for KCOPARKE331: Success
Retrieving data for KCOPARKE298: Success
Retrieving data for KCOPARKE414: Success
Retrieving data for KCOPARKE374: Success
Retrieving data for KCOPARKE320: Success
Retrieving data for KCOPARKE26: Success
Retrieving data for KCOPARKE276: Success
Retrieving data for KCOPARKE303: Success
Retrieving data for KCOPARKE271: Success
Retrieving data for KCOPARKE148: Success
Retrieving data for KCOPARKE201: Success
Retrieving data for KCOPARKE34: Success
Retrieving data for KCOPARKE313: Success
Retrieving data for KCOPARKE343: Success
Retrieving data for KCOPARKE278: Success
Retrieving data for KCOPARKE305: Success
Retrieving data for KCOCASTL161: Success
Retrieving data for KCOPARKE68: Success
Retrieving data for KCOPARKE399: Success
Retrieving data for KCOPARKE386: Success
Retrieving data for KCOCASTL227: Success
Retrieving data for KCOPARKE176: Success
Retrieving data for KCOPARKE256: Success
Retrieving data for KCOPARKE245: Success
Retrieving data for

In [ ]:
observations_df.drop(columns=drop_columns,inplace=True)
observations_df.to_csv('/content/drive/MyDrive/Capstone/PSDS4900/wu_data/{}.csv'.format(datetime.now().date().strftime('%Y%m%d')),index=False)
print('/content/drive/MyDrive/Capstone/PSDS4900/wu_data/{}.csv written\n'.format(datetime.now().date().strftime('%Y%m%d')))  

/content/drive/MyDrive/Capstone/PSDS4900/wu_data/20210331.csv written

